In [3]:
# Re-import pandas for the current environment
import pandas as pd

# Reload the data files
customer_data = pd.read_csv("data/customer_data.csv")
usage_data = pd.read_csv("data/usage_data.csv")

# Display the first few rows of both datasets to understand their structure
customer_data.head(), usage_data.head()


C:\Users\sabal\AppData\Local\Temp\ipykernel_10904\561792766.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  usage_data = pd.read_csv("data/usage_data.csv")


(         customerid product_name signup_date channel first_activation_date  \
 0  9059978095643230   QuickBooks     6/21/21     PPC               6/21/21   
 1  9059978095913820    Mailchimp     6/25/21     PPC               6/26/21   
 2  9130350795302030          NaN      6/1/21     PPC                   NaN   
 3  9130350795888320          NaN      6/1/21  Direct                   NaN   
 4  9130350796379850    Mailchimp      6/1/21  Direct                7/1/21   
 
   first_purchase_date cancel_date Unnamed: 7  
 0                 NaN         NaN        NaN  
 1                 NaN         NaN        NaN  
 2                 NaN         NaN        NaN  
 3                 NaN         NaN        NaN  
 4              7/9/21     8/10/21        NaN  ,
   customerid product_name event_date  action_type_id  usage_count
 0  116490232     TurboTax     9/3/21               5            1
 1  116490232     TurboTax    9/14/21               7            4
 2  116490232     TurboTax    9/16

In [6]:
# Active users (First_Activation_Date is not null) grouped by Product_Name
active_users = customer_data[~customer_data['first_activation_date'].isna()]
active_users_by_product = active_users.groupby('product_name').size()

# Churned users (Cancel_Date is not null) grouped by Product_Name
churned_users = customer_data[~customer_data['cancel_date'].isna()]
churned_users_by_product = churned_users.groupby('product_name').size()

# Combine results into a summary dataframe
kpi_summary = pd.DataFrame({
    "Active_Users": active_users_by_product,
    "Churned_Users": churned_users_by_product
}).fillna(0).astype(int)
kpi_summary

,Active_Users,Churned_Users
product_name,,
Mailchimp,2059,675
Mint,2043,612
QuickBooks,2145,682
TurboTax,1994,616


In [12]:
# Filter customers who have a signup date but no first activation date
customers_without_activation = customer_data[
    customer_data['signup_date'].notnull() & customer_data['first_activation_date'].isnull() & customer_data['product_name'].notnull()
]

# Display the count of such customers
num_customers_without_activation = customers_without_activation.shape[0]

print(f"Number of customers signed up without a first activation date: {num_customers_without_activation}")

# Optionally, display the subset of the data
customers_without_activation.shape


Number of customers signed up without a first activation date: 0


(0, 8)

In [14]:
active_users_by_product = customer_data[customer_data['cancel_date'].isnull()]\
    .groupby('product_name')['customerid'].nunique()\
    .reset_index()

active_users_by_product.columns = ['Product', 'Active Users']

active_users_by_product

,Product,Active Users
0,Mailchimp,1337
1,Mint,1392
2,QuickBooks,1413
3,TurboTax,1335


In [19]:
usage_data['event_date'].unique()
usage_data['event_date']= pd.to_datetime(usage_data['event_date'], errors='coerce')
usage_data['event_date'].min()

Timestamp('2021-06-01 00:00:00')

In [20]:
usage_data['event_date'].max()

Timestamp('2021-09-28 00:00:00')

In [ ]:
customer_data['signup_date']= pd.to_datetime(customer_data['signup_date'],errors='coerce')
customer_data['signup_date'].max()


Timestamp('2021-06-30 00:00:00')

In [32]:
customer_data["first_activation_date"] = pd.to_datetime(customer_data["first_activation_date"], errors='coerce')
customer_data["cancel_date"] = pd.to_datetime(customer_data["cancel_date"], errors='coerce')

# Recalculate start and end dates
start_date = min(
    customer_data["first_activation_date"].min(),
    customer_data["cancel_date"].min()
)
end_date = max(
    customer_data["first_activation_date"].max(),
    customer_data["cancel_date"].max()
)

# Generate a full date range
full_date_range = pd.date_range(start=start_date, end=end_date, freq="D")

# Filter data for Mailchimp
mailchimp_data = customer_data[customer_data["product_name"] == "Mailchimp"]

# Recalculate cumulative activated customers for Mailchimp
cumulative_activated_customers_mailchimp = (
    mailchimp_data.groupby("first_activation_date").size()
    .reindex(full_date_range, fill_value=0)
    .cumsum()
)

# Recalculate cumulative cancelled customers for Mailchimp
cumulative_cancelled_customers_mailchimp = (
    mailchimp_data.groupby("cancel_date").size()
    .reindex(full_date_range, fill_value=0)
    .cumsum()
)

# Recalculate active customers for Mailchimp
active_customers_daily_mailchimp = cumulative_activated_customers_mailchimp - cumulative_cancelled_customers_mailchimp

# Validate final values
lifetime_activated_mailchimp = cumulative_activated_customers_mailchimp.iloc[-1]
current_active_mailchimp = active_customers_daily_mailchimp.iloc[-1]

lifetime_activated_mailchimp, current_active_mailchimp


(np.int64(2059), np.int64(1384))

In [31]:
current_active_mailchimp

np.int64(1384)